In [61]:
import pandas as pd
import numpy as np
import os
import csv
import shutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [62]:
# Define the directory paths
dir_paths = ["Dataset/ECG Images of COVID-19 Patients (250)", "Dataset/ECG Images of Myocardial Infarction Patients (77)", "Dataset/ECG Images of Patient that have abnormal heart beats (548)", "Dataset/ECG Images of Patient that have History of MI (203)", "Dataset/Normal Person ECG Images (859)"]

# Create a list to store the image filenames
image_filenames = []

# Loop through the directories and add the image filenames to the list
for dir_path in dir_paths:
    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_filenames.append(filename)

# Create a CSV file and write the image filenames and label "0" to it
with open("image_labels.csv", mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["filename", "label"])
    for image_filename in image_filenames:
        if "Normal" in image_filename:
            writer.writerow([image_filename, "0"])
        else:
            writer.writerow([image_filename, "1"])

In [63]:
import os
import cv2

# directory containing the images to be cropped
input_dir = "Dataset/ECG Images of COVID-19 Patients (250)/"

# loop through all files in the directory
for filename in os.listdir(input_dir):
    # check if the file is an image file
    if filename.endswith(".jpg"):
        # read the image
        img = cv2.imread(os.path.join(input_dir, filename))
        
        # perform the cropping operation
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
        cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        cnt = sorted(cnts, key=cv2.contourArea)[-1]
        x,y,w,h = cv2.boundingRect(cnt)
        dst = img[y:y+h, x:x+w]
        
        # save the cropped image with the same filename
        cv2.imwrite(os.path.join(input_dir, filename), dst)


In [64]:
# define paths
source_folder = 'Dataset'
destination_folder = 'images'

# create destination folder if not exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# loop through subdirectories
for subdir in os.listdir(source_folder):
    subdir_path = os.path.join(source_folder, subdir)
    
    # check if it is a directory and not a file
    if os.path.isdir(subdir_path) and "ECG" in subdir_path:
        print(subdir)
        
        # loop through files in subdirectory
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            
            # copy file to destination folder
            shutil.copy(file_path, destination_folder)

ECG Images of COVID-19 Patients (250)
ECG Images of Patient that have History of MI (203)
ECG Images of Patient that have abnormal heart beats (548)
Normal Person ECG Images (859)
ECG Images of Myocardial Infarction Patients (77)


In [65]:
# Set random seed for reproducibility
np.random.seed(42)

# Load CSV file containing image filenames and labels
df = pd.read_csv('image_labels.csv')
df["label"] = df["label"].astype(str)

In [66]:
# Split dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [67]:
# Create directories for train, validation, and test sets
train_dir = 'train'
os.makedirs(train_dir, exist_ok=True)

test_dir = 'test'
os.makedirs(test_dir, exist_ok=True)

In [68]:
# Move images to respective directories based on the split
for i, row in train_df.iterrows():
    src = 'images/' + row['filename']
    dst = train_dir + '/' + row['filename']
    shutil.copyfile(src, dst)

for i, row in test_df.iterrows():
    src = 'images/' + row['filename']
    dst = test_dir + '/' + row['filename']
    shutil.copyfile(src, dst)

In [69]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=preprocess_input)

# No data augmentation for testing set, only rescaling
test_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=preprocess_input)


In [70]:
# Define batch size
batch_size = 32

# Set target image size
target_size = (224, 224)

In [71]:
# Create training and testing generators
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory=train_dir,
                                                    x_col='filename',
                                                    y_col='label',
                                                    target_size=target_size,
                                                    batch_size=batch_size,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  directory=test_dir,
                                                  x_col='filename',
                                                  y_col='label',
                                                  target_size=target_size,
                                                  batch_size=batch_size,
                                                  class_mode='binary')

Found 1541 validated image filenames belonging to 2 classes.
Found 386 validated image filenames belonging to 2 classes.
